<h1>New restaurant in Moscow.</h1>

<h3>March, 2019</h3>

<h2>I.	Introduction</h2>

<p>Moscow is a highly developing city. One of tendencies for last 2-3 years is a rapid development and constriction of areas on the outskirts of the city, especially Northwest Administrative District. </p>
<p>And new business centers and residential areas means that the district needs more social venues, including restaurants and cafés.</p>
<p>In this project I’ll evaluate all existing venues within Northwest Administrative District and try to predict the best area for opening the new restaurant. As it’s not the city center and the district is still developing the competition should be lower as well as the rental price for next few years when the expenses of the new business are the highest.</p>


<h2>II.	Data description.</h2>

<p>I used Foursquare to get the list of venues in Moscow. </p>
<p>There are no public data for geo coordinates of the district so I had to create my own base.  I used Google Maps to find the center of each area including to the Northwest Administrative District.</p>

<h2>III.	Methodology description.</h2>

<p>For database of areas within the chosen district I created my own dataframe (using Google Maps for coordinates). </p>

In [283]:
#import libraties
import numpy as np 
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json 
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 

from pandas.io.json import json_normalize 
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes 
import folium 

#set Fooursquare credentials
CLIENT_ID = 'YB3J2EUQC3VKPOTICQVQBHQNABLSBH0NR1FSMCOCAGRVT5DG' 
CLIENT_SECRET = 'VCUJLBIG2WEATY1IFG42FETVXVEB45HTHOK3RY2CNOBAXPWW'
VERSION = '20180605' 
print('Libraries imported')

Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
geopy                     1.18.1                     py_0    conda-forge
Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
folium                    0.5.0                      py_0    conda-forge
Libraries imported


In [284]:
#Create the additional df for chosenMoscow region.

mdist = {'District': ['Kurkino','Mitino', 'Pokrovskoe-Streshnevo', 'Strogino', 'North Tushino', 'South Tushino', 'Khoroshovo-Mnevniki', 'Shchukino'
], 'Latitude':[55.891917, 55.850973, 55.821969, 55.789169, 55.859291, 55.845282, 55.774161, 55.802158], 'Longitude': [37.388824, 37.366766, 37.448387, 
37.394399, 37.425743, 37.428557, 37.473079, 37.467494]}

df_mdist = pd.DataFrame(data=mdist)
df_mdist.head(10)

,District,Latitude,Longitude
0,Kurkino,55.891917,37.388824
1,Mitino,55.850973,37.366766
2,Pokrovskoe-Streshnevo,55.821969,37.448387
3,Strogino,55.789169,37.394399
4,North Tushino,55.859291,37.425743
5,South Tushino,55.845282,37.428557
6,Khoroshovo-Mnevniki,55.774161,37.473079
7,Shchukino,55.802158,37.467494


In [285]:
#set Moscow on the map
address = 'Волоколамское ш., 69, Москва'
geolocator = Nominatim(user_agent="moscow_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

<p>I used python folium library to visualize areas on the map using latitude and longitude.</p>

In [286]:
#create a map for the selected region
map_moscow = folium.Map(location=[latitude, longitude], zoom_start=12)
for lat, lng, district in zip(df_mdist['Latitude'], df_mdist['Longitude'], df_mdist['District']):
    label = '{}'.format(district)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=7,
        popup=label,
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=0.9,
        parse_html=False).add_to(map_moscow)  
    
map_moscow

<p>Then I used Foursquare database to extract all venues by areas. I excluded from Categories the ‘Food’ (I want to see it on the separate map) and ‘Colleges and Universities‘ (I believe they will not have a great impact on the demand). </p>

In [287]:
#creare a function to return venues from all districts for all venues except Food            
import requests
#4d4b7104d754a06370d81259 - Искусство и развлечения add
#4d4b7105d754a06372d81259-  Высшее учебное заведение exclude
#4d4b7105d754a06373d81259 - Событие add
#4d4b7105d754a06376d81259 - Ночное заведение add
#4d4b7105d754a06377d81259 - Заведения на свежем воздухе add
#4d4b7105d754a06375d81259 - Услуги и учреждения add
#4d4b7105d754a06378d81259 - Магазины и услуги
#4d4b7105d754a06379d81259 - Путешествия и транспорт

def getNearbyVenues(names, latitudes, longitudes, radius=2000, LIMIT=1000, categoryId='4d4b7104d754a06370d81259,4d4b7105d754a06373d81259,4d4b7105d754a06376d81259,4d4b7105d754a06377d81259,4d4b7105d754a06375d81259,4d4b7105d754a06378d81259,4d4b7105d754a06379d81259'):
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId={}'.format(
            CLIENT_ID,CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT, categoryId)
        results = requests.get(url).json()["response"]['groups'][0]['items']
        venues_list.append([(
            name, lat, lng, v['venue']['name'], v['venue']['location']['lat'], v['venue']['location']['lng'], v['venue']['categories'][0]['name']) for v in results])
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['District', 
                  'District Latitude', 'District Longitude', 'Venue', 'Venue Latitude', 'Venue Longitude', 'Venue Category']    
    return(nearby_venues)

In [288]:
moscow_venues = getNearbyVenues(names=mdist['District'],
                                   latitudes=mdist['Latitude'],
                                   longitudes=mdist['Longitude']
                                  )
moscow_venues.head()

,District,District Latitude,District Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Kurkino,55.891917,37.388824,Modulus,55.894698,37.389286,Dance Studio
1,Kurkino,55.891917,37.388824,Melior,55.895828,37.391191,Dance Studio
2,Kurkino,55.891917,37.388824,Todes,55.897382,37.389333,Dance Studio
3,Kurkino,55.891917,37.388824,"Центр Творчества И Обучения ""Верный Курс""",55.896126,37.396305,Performing Arts Venue
4,Kurkino,55.891917,37.388824,Coral travel,55.897500,37.405043,Tour Provider


<p>I drop duplicates and get the total number of venues within each area, in total – 77 venue for 7 areas.</p>

In [289]:
#delete duplicate rows
moscow_venues.drop_duplicates(keep=False, inplace=True)
moscow_venues[['District', 'Venue Category']].groupby("District").count()

,Venue Category
District,
Khoroshovo-Mnevniki,13
Kurkino,6
Mitino,9
North Tushino,10
Pokrovskoe-Streshnevo,10
Shchukino,18
South Tushino,8
Strogino,3


In [290]:
print('There are ', moscow_venues.shape[0], 'venues for ', moscow_venues.shape[1], ' areas.')

There are  77 venues for  7  areas.


<p>Now I do the same for category ‘Food’.</p>

In [291]:
#creare a function to return venues from all districts for Food            
import requests

def getNearbyVenues_food(names, latitudes, longitudes, radius=2000, LIMIT=1000, categoryId='4d4b7105d754a06374d81259'):    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT,
            categoryId)
        results = requests.get(url).json()["response"]['groups'][0]['items']
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['District', 
                  'District Latitude', 
                  'District Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']    
    return(nearby_venues)

In [ ]:
moscow_venues_food = getNearbyVenues_food(names=mdist['District'],
                                   latitudes=mdist['Latitude'],
                                   longitudes=mdist['Longitude'])
moscow_venues_food.head()

In [ ]:
moscow_venues_food.drop_duplicates(keep=False, inplace=True)
moscow_venues_food[['District', 'Venue Category']].groupby("District").count()

<p>To visualize how all venues are spread within areas I used the Folium library and create a new map, for Food venues I used the purple color, for all other venues – the pink color. </p>
<p>Even from the brief look it’s clear that there are already lot’s of different ‘Food’ venues and so much infrastructure objects which can generate the high demand, so the competition could be quite rough.
</p>

In [ ]:
#create a map for the selected region
map_moscow = folium.Map(location=[latitude, longitude], zoom_start=12)

for lat, lng, venue in zip(moscow_venues['Venue Latitude'], moscow_venues['Venue Longitude'], moscow_venues['Venue']):
    label = '{}'.format(venue)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color='deeppink',
        fill=False,
        fill_color='deeppink',
        fill_opacity=0.7,
        parse_html=False).add_to(map_moscow)  
    
for lat, lng, venue in zip(moscow_venues_food['Venue Latitude'], moscow_venues_food['Venue Longitude'], moscow_venues_food['Venue']):
    label = '{}'.format(venue)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color='indigo',
        fill=False,
        fill_color='indigo',
        fill_opacity=0.7,
        parse_html=False).add_to(map_moscow) 
    
map_moscow

<p>For further analysis I’m using k-mean clustering. I used k=4, also I added top-5 venues for each area. </p>

In [ ]:
# one hot encoding
moscow_onehot = pd.get_dummies(moscow_venues[['Venue Category']], prefix="", prefix_sep="")
moscow_onehot['District'] = moscow_venues['District'] 
fixed_columns = [moscow_onehot.columns[-1]] + list(moscow_onehot.columns[:-1])
moscow_onehot = moscow_onehot[fixed_columns]
moscow_grouped = moscow_onehot.groupby('District').mean().reset_index()

num_top_venues = 3

for hood in moscow_grouped['District']:
    temp = moscow_grouped[moscow_grouped['District'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})

In [ ]:
#convert into dataframe to see top venues
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False) 
    return row_categories_sorted.index.values[0:num_top_venues]
    
num_top_venues = 5
indicators = ['st', 'nd', 'rd']
columns = ['District']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['District'] = moscow_grouped['District']

for ind in np.arange(moscow_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(moscow_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head(8)

In [ ]:
#cluster districts
kclusters = 4
moscow_grouped_clustering = moscow_grouped.drop('District', 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(moscow_grouped_clustering)
kmeans.labels_[0:10] 
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
moscow_merged = df_mdist
moscow_merged = moscow_merged.join(neighborhoods_venues_sorted.set_index('District'), on='District')
moscow_merged.head(8) 

<h3>Cluster 1 - Strogino, Kurkino.</h3>

In [ ]:
moscow_merged.loc[moscow_merged['Cluster Labels'] == 0, moscow_merged.columns[[0] + list(range(4, moscow_merged.shape[1]))]]

<h3>Cluster 2 - North Tushino, South Tushino.</h3>

In [ ]:
moscow_merged.loc[moscow_merged['Cluster Labels'] == 1, moscow_merged.columns[[0] + list(range(4, moscow_merged.shape[1]))]]

<h3>Cluster 3 - Pokrovskoe-Streshnevo, Khoroshovo-Mnevniki, Shchukino.</h3>

In [ ]:
moscow_merged.loc[moscow_merged['Cluster Labels'] == 2, moscow_merged.columns[[0] + list(range(4, moscow_merged.shape[1]))]]

<h3>Cluster 4 - Mitino.</h3>

In [ ]:
moscow_merged.loc[moscow_merged['Cluster Labels'] == 3, moscow_merged.columns[[0] + list(range(4, moscow_merged.shape[1]))]]

<h2>IV.	Results.</h2>

<p>I created the new map to visualize clusters and the existing venues.</p>

In [ ]:
#create map with clusters
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(moscow_merged['Latitude'], moscow_merged['Longitude'], moscow_merged['District'], moscow_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=25,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.2).add_to(map_moscow)

map_moscow

<p>From this map and evaluation of each clusters I can give the recommendation to open the restaurant of the Cluster 3 (areas Pokrovskoe-Streshnevo, Khoroshovo-Mnevniki, Shchukino). Among the top-values are enough popular places such as museums, galleries etc. Comparing with other cluster it’s not so far from the city center, which may give additional benefits.   </p>

<h2>V.	Discussions.</h2>

<p>As it was mentioned in the beginning this research was based only on data of the one district in Moscow - Northwest Administrative District which I believe to be the fastest growing. For better predictions it’s necessary to get information for the whole city as well.</p>
<p>For the more proper research the potential investor should know the type of the restaurant he plans to open and the average pricing so all other categories could be dropped in Foursquare (or other sources). </p>
<p>Another point – for venues search it was used only Foursquare API source. The quantity of Non-Food venues which I get looks quite small for the area which means that probably not all mentioned on the Foursquare database. It means that it should be used additional statistical and geo databases (probably Google Nearby Search or some local sources) to get more results.</p>
<p>And the last but not least point is to have the statistical data for the profitability of existing competitors among areas and use it for clustering and segmentation as well, as it’s a key point of each business work. </p>

<h2>VI. Conclusion.</h2>

<p>Making such type of the research should be the first thing which the investor do before creating any new business. It shows in the very simple way (even for someone without deep business knowledge) all main aspects which should be considered – the potential demand (and where it comes from), potential profitability and competitors. </p>

<h2>VII.	References.</h2>

<ul>
    <li><a href = "https://developer.foursquare.com/docs">Foursquare API.</a></li>
    <li><a href = "https://developers.google.com/maps/documentation/geocoding/intro">Google maps for developers.</a></li>
    <li><a href = "http://mosopen.ru/regions">Moscow districts.</a></li>